In [1]:
# Step 0 - Install Libraries

!pip install selenium pandas beautifulsoup4



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\nurul\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
# Step 1 – Import libraries

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time


In [3]:
# Step 2 – Setup WebDriver
options = Options()
# options.add_argument('--headless')  # Aktifkan kalau mau tanpa buka browser
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(
    service=Service("C:/Users/nurul/Downloads/chromedriver-win64/chromedriver-win64/chromedriver.exe"),
    options=options
)


In [4]:
# Step 3 – Kategori yang diambil (tech-related)
kategori_links = {
    "Data & Product": "https://dealls.com/?loker=data-and-product",
    "Design & Creative": "https://dealls.com/?loker=design-and-creative",
    "IT & Engineering": "https://dealls.com/?loker=it-and-engineering",
    "Marketing & Social Media": "https://dealls.com/?loker=marketing-and-social-media"
}

all_data = []

In [5]:
for kategori, url in kategori_links.items():
    print(f"\n🌐 Memproses kategori: {kategori}")
    driver.get(url)
    time.sleep(2)

    # Step 4 – Klik tombol "Lebih Banyak"
    clicks = 0
    max_clicks = 25
    while clicks < max_clicks:
        try:
            load_more_btn = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.XPATH, "//button[.//span[contains(text(), 'Lebih Banyak')]]"))
            )
            driver.execute_script("arguments[0].scrollIntoView(true);", load_more_btn)
            time.sleep(1)
            load_more_btn.click()
            clicks += 1
            print(f"🔁 Klik 'Lebih Banyak' ke-{clicks}")
            time.sleep(2)
        except Exception as e:
            print(f"✅ Selesai klik atau tombol tidak ditemukan: {e}")
            break

    # Step 5 – Ambil semua card job
    job_cards = driver.find_elements(By.XPATH, "//a[starts-with(@href, '/loker/')]")

    for card in job_cards:
        try:
            job_title_element = card.find_element(By.XPATH, ".//h2 | .//div[contains(@class,'font-bold')]")
            job_title = job_title_element.text.strip()
        except:
            continue

        detail_link = card.get_attribute("href")

        try:
            company = card.find_element(By.XPATH, ".//div[contains(@class,'gap-[4.6px]')]").text.strip().split("\n")[0]
        except:
            company = ""

        try:
            work_type = card.find_element(By.XPATH, ".//div[contains(@class,'text-tertiary-violet-50')]").text.strip()
        except:
            work_type = ""

        try:
            infos = card.find_elements(By.XPATH, ".//div[contains(@class,'text-neutral-100')]//span")
            work_mode = infos[0].text.strip() if len(infos) > 0 else ""
            experience = infos[1].text.strip() if len(infos) > 1 else ""
            salary = infos[2].text.strip() if len(infos) > 2 else ""
        except:
            work_mode = experience = salary = ""

        all_data.append({
            "Kategori": kategori,
            "Job Title": job_title,
            "Company": company,
            "Work Type": work_type,
            "Work Mode": work_mode,
            "Experience": experience,
            "Salary": salary,
            "Detail Link": detail_link
        })

print(f"\n📌 Total job dikumpulkan: {len(all_data)}")



🌐 Memproses kategori: Data & Product
🔁 Klik 'Lebih Banyak' ke-1
🔁 Klik 'Lebih Banyak' ke-2
🔁 Klik 'Lebih Banyak' ke-3
✅ Selesai klik atau tombol tidak ditemukan: Message: 
Stacktrace:
	GetHandleVerifier [0x0x7ff7c99d6f75+76917]
	GetHandleVerifier [0x0x7ff7c99d6fd0+77008]
	(No symbol) [0x0x7ff7c9789dea]
	(No symbol) [0x0x7ff7c97e0256]
	(No symbol) [0x0x7ff7c97e050c]
	(No symbol) [0x0x7ff7c9833887]
	(No symbol) [0x0x7ff7c98084af]
	(No symbol) [0x0x7ff7c983065c]
	(No symbol) [0x0x7ff7c9808243]
	(No symbol) [0x0x7ff7c97d1431]
	(No symbol) [0x0x7ff7c97d21c3]
	GetHandleVerifier [0x0x7ff7c9cad2ad+3051437]
	GetHandleVerifier [0x0x7ff7c9ca7903+3028483]
	GetHandleVerifier [0x0x7ff7c9cc589d+3151261]
	GetHandleVerifier [0x0x7ff7c99f183e+185662]
	GetHandleVerifier [0x0x7ff7c99f96ff+218111]
	GetHandleVerifier [0x0x7ff7c99dfaf4+112628]
	GetHandleVerifier [0x0x7ff7c99dfca9+113065]
	GetHandleVerifier [0x0x7ff7c99c6c78+10616]
	BaseThreadInitThunk [0x0x7ffd6486259d+29]
	RtlUserThreadStart [0x0x7ffd6600a

In [6]:
# Step 6 – Ambil detail deskripsi & kualifikasi
job_detail_data = []

for idx, job in enumerate(all_data):
    print(f"🔍 Scraping detail {idx+1}/{len(all_data)}: {job['Job Title']}")
    try:
        driver.get(job["Detail Link"])
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(
                (By.XPATH, "//h3[contains(text(), 'Deskripsi Pekerjaan') or contains(text(),'Kualifikasi') or contains(text(),'Job Requirements')]")
            )
        )

        # Deskripsi
        desc_parts = []
        try:
            desc_blocks = driver.find_elements(By.XPATH, "//h3[contains(text(), 'Deskripsi Pekerjaan')]/following-sibling::*")
            for elem in desc_blocks:
                if elem.tag_name == "h3":
                    break
                ps = elem.find_elements(By.TAG_NAME, "p")
                if ps:
                    desc_parts.extend([p.text.strip() for p in ps if p.text.strip()])
                else:
                    text = elem.text.strip()
                    if text:
                        desc_parts.append(text)
        except:
            pass
        description = "\n".join(desc_parts)

        # Kualifikasi
        qualif_parts = []
        try:
            qualif_blocks = driver.find_elements(By.XPATH, "//h3[contains(text(), 'Kualifikasi') or contains(text(), 'Job Requirements')]/following-sibling::*")
            for elem in qualif_blocks:
                if elem.tag_name == "h3":
                    break
                lis = elem.find_elements(By.TAG_NAME, "li")
                if lis:
                    qualif_parts.extend([li.text.strip() for li in lis if li.text.strip()])
                else:
                    text = elem.text.strip()
                    if text:
                        qualif_parts.append(text)
        except:
            pass
        qualification = "\n".join(qualif_parts)

        # Benefit Perusahaan
        benefits = []
        try:
            benefit_blocks = driver.find_elements(By.XPATH, "//h3[contains(text(), 'Benefit Perusahaan')]/following-sibling::ul[1]/li")
            benefits = [b.text.strip() for b in benefit_blocks if b.text.strip()]
        except:
            pass
        benefit_text = "\n".join(benefits)


        job_detail_data.append({
            **job,
            "Job Description": description,
            "Qualification": qualification,
            "Benefit": benefit_text
        })

    except Exception as e:
        print(f"❌ Gagal ambil detail untuk {job['Job Title']} - {e}")


🔍 Scraping detail 1/901: Product - Director
🔍 Scraping detail 2/901: Product Planning - Staff
🔍 Scraping detail 3/901: Strategic Sourcing - Officer
🔍 Scraping detail 4/901: Process Management System Analyst
🔍 Scraping detail 5/901: Measurement Product - Specialist
🔍 Scraping detail 6/901: Business Analyst - Indonesia
🔍 Scraping detail 7/901: Platinion Senior IT - Architect
🔍 Scraping detail 8/901: Consultant, Technology & Digital Advantage - Practice
🔍 Scraping detail 9/901: Quality Control - Staff
🔍 Scraping detail 10/901: Manufacture Development - Program
🔍 Scraping detail 11/901: Packaging & Device Development - Officer
🔍 Scraping detail 12/901: Lab Production Staff - Pharmacist
🔍 Scraping detail 13/901: Credit Operations Analyst (English Speaker)
🔍 Scraping detail 14/901: Promotor Provider Ternama ( Jambi )
🔍 Scraping detail 15/901: Bussines Analyst
🔍 Scraping detail 16/901: Data Center Facility Engineer (L2 Specialist)
🔍 Scraping detail 17/901: Retail Supervisor Area Papua (Merauk

In [11]:
# Step 6 – Simpan CSV
df_detail = pd.DataFrame(job_detail_data)
df_detail.to_csv("dealls_jobs_tech.csv", index=False)
print("\n✅ Selesai! Data disimpan ke dealls_jobs_tech.csv")


✅ Selesai! Data disimpan ke dealls_jobs_tech.csv


In [12]:
df_detail.tail(20)

,Kategori,Job Title,Company,Work Type,Work Mode,Experience,Salary,Detail Link,Job Description,Qualification,Benefit
880,Marketing & Social Media,Digital Ads Specialist,PT Qtera Mandiri,Penuh waktu,On-site • Jakarta Utara,Min. Freshgrad,Negotiable,https://dealls.com/loker/digital-ads-specialis...,Tanggung Jawab:,"Minimal S1 dari jurusan Marketing, Komunikasi,...",THR / Bonus system\nTeam Building Activity
881,Marketing & Social Media,Social Media Specialist (Surabaya),Vista Education Corp.,Penuh waktu,On-site • Surabaya,Min. Freshgrad,Rp5.000.000 – 5.500.000,https://dealls.com/loker/social-media-speciali...,Requirements:,Copywriting: Ability to create engaging captio...,Competitive Salary\nMedical Insurance\nTHR / B...
882,Marketing & Social Media,Freelance Content Writer,Tim Jess No Limit & Sisca Kohl,Freelance,Remote,Min. 1 years of experience,Negotiable,https://dealls.com/loker/freelance-content-wri...,Job Description:\nWe are looking for a creativ...,Strong interest and knowledge in gaming cultur...,THR / Bonus system\nMedical Insurance\nCasual ...
883,Marketing & Social Media,KOL Specialist,TXT Creative,Kontrak,On-site • Tangerang Regency,Min. 1 years of experience,Negotiable,https://dealls.com/loker/kol-specialist-54~txt...,"Identify, evaluate, and onboard relevant KOLs ...","Minimum 2–3 years of experience in KOL, influe...",Competitive Salary\nMedical Insurance\nTHR / B...
884,Marketing & Social Media,Social Media Specialist,Evolene Group,Penuh waktu,On-site • Tangerang,Min. 1 years of experience,Rp5.000.000 – 7.000.000,https://dealls.com/loker/social-media-speciali...,Assist with content for social media and marke...,Atleast 1 year of experience.\nGood at talking...,THR / Bonus system\nCompetitive Salary\nMedica...
885,Marketing & Social Media,Marketing Admin,Delta Systech Indonesia,Kontrak,On-site • Tangerang Selatan,Min. Freshgrad,Negotiable,https://dealls.com/loker/marketing-admin-2~del...,"Merapikan, mengarsipkan, dan mengelola dokumen...",Pendidikan minimal SMK atau sederajat (diutama...,THR / Bonus system\nProfessional Development\n...
886,Marketing & Social Media,KOL Specialist Intern,Felaufee,Magang,On-site • Jakarta Utara,Min. 3rd Year College Student,Rp2.000.000 – 2.500.000,https://dealls.com/loker/kol-specialist-intern...,"Assist in all phases of KOL activities, from r...",Final-year students or recent graduates\nHighl...,Competitive Salary\nPet-friendly Office\nPaid ...
887,Marketing & Social Media,Social Media Specialist,Felaufee,Penuh waktu,On-site • Jakarta Utara,Min. 1 years of experience,Rp6.000.000 – 9.000.000,https://dealls.com/loker/social-media-speciali...,Develop visually engaging and high-quality con...,"1+ year of experience as a content creator, wi...",Competitive Salary\nPet-friendly Office\nPaid ...
888,Marketing & Social Media,Content Producer,PT Kolaborasi Mediapreneur Nusantara (Pikiran ...,Penuh waktu,On-site • Bandung,Min. 1 years of experience,Rp4.000.000 – 4.482.914,https://dealls.com/loker/content-producer-2~pt...,"Bertanggung jawab dalam merancang, mengembangk...","Pengalaman dalam bidang content production, co...",Paid Sick Days\nCompetitive Salary\nTHR / Bonu...
889,Marketing & Social Media,Online Growth Marketing,UENA Food,Penuh waktu,On-site • Jakarta Barat,Min. Freshgrad,Negotiable,https://dealls.com/loker/online-growth-marketi...,"Where you will tackle some challenges, such as :","Bachelor’s degree in Marketing, Business, or r...",Competitive Salary\nTHR / Bonus system\nPaid S...


In [9]:
df_detail.shape

(900, 11)

In [14]:
# Step 5 - Close Browser
driver.quit()
